In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from PIL import Image
import requests
from io import BytesIO
import os
import zipfile
import re
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
from selenium.webdriver.chrome.options import Options
import urllib.request
import shutil


download_path = '/Users/jonathanoh/Desktop/down'
script_dir = '/Users/jonathanoh/Desktop/script/crawling'
s = Service('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# script_dir = "/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/crawling"
# download_path = "/Users/jonathanoh/Desktop/contents/exiftoexcel/temp_image_zip_treated"
# s = Service('/Users/jonathanoh/Desktop/contents/exiftoexcel/스크립트/chromedriver-mac-arm64/chromedriver')





def combine_csv_files_remove_duplicates(directory_path):
    combined_df = pd.DataFrame()

    # List to store individual dataframes
    dfs = []

    # Iterate through each file in the directory
    for file in os.listdir(directory_path):
        
        if file.endswith('.csv'):
            file_path = os.path.join(directory_path, file)
            # Read each CSV file
            df = pd.read_csv(file_path)
            while len(df.columns)>2:
                del df[df.columns[-1]]
            # Append the dataframe to the list
            dfs.append(df)
            
#     print(dfs)
    # Combine all dataframes in the list
    combined_df = pd.concat(dfs, ignore_index=True)
    if dfs:
        combined_df.columns=['url', 'filename']
    
    for file in os.listdir(directory_path):
        if file.endswith('.csv') :
            os.remove(os.path.join(directory_path, file))
    csv_file = os.path.join(directory_path, "url_data.csv")

    
    combined_df.to_csv(csv_file, index=False)
    
    # Remove duplicates
    combined_df.drop_duplicates(inplace=True)
    
#     for file in os.listdir(directory_path):
#         if file.endswith('.csv'):
            
    return combined_df



def zip_folder(folder_path, zip_name):
    # 폴더를 ZIP 파일로 압축
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file), 
                           os.path.relpath(os.path.join(root, file), 
                                           os.path.join(folder_path, '..')))

def compress_and_delete_folders(base_path):
    for item in os.listdir(base_path):
        item_path = os.path.join(base_path, item)
        if os.path.isdir(item_path):
            zip_name = item_path + '.zip'
            zip_folder(item_path, zip_name)
            shutil.rmtree(item_path)
            print(f"Compressed and removed folder: {item}")



def login_to_website(driver, username, password):
    driver.get('https://arca.live/u/login?goto=%2Fb%2Faiartreal')

    username_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'username'))
    )
    username_input.send_keys(username)

    login_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'submitBtn'))
    )
    login_button.click()

    password_input = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.NAME, 'password'))
    )
    password_input.send_keys(password)
    login_button.click()


# def fetch_post_list(driver):
#     try:
#         WebDriverWait(driver, 10).until(
#             EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
#         )

#         post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")

#         posts = []
#         for post_element in post_elements:
#             href = post_element.get_attribute('href')
#             try:
#                 title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
#                 title = title_element.text
#             except Exception:
#                 title = "제목을 찾을 수 없음"
#             posts.append((href, title))

#         return posts

#     except Exception as e:
#         print(f"에러 발생: {e}")
#         return []

def fetch_post_list(driver):
    try:
        WebDriverWait(driver, 2).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a.vrow.column"))
        )

        post_elements = driver.find_elements(By.CSS_SELECTOR, "a.vrow.column")
        posts = []

        for post_element in post_elements:
            href = post_element.get_attribute('href')
            title = "제목을 찾을 수 없음"
            user = "사용자를 찾을 수 없음"

            try:
                title_element = post_element.find_element(By.CSS_SELECTOR, "span.title")
                title = title_element.text if title_element else title
            except Exception:
                pass  # 제목을 찾을 수 없는 경우 기본값 사용

            try:
                user_element = post_element.find_element(By.CSS_SELECTOR, 'span[data-filter]')
                user = user_element.text if user_element else user
            except Exception:
                pass  # 사용자를 찾을 수 없는 경우 기본값 사용

            posts.append((href, title, user))

        return posts

    except Exception as e:
        print(f"에러 발생: {e}")
        return []


# def compress_images_to_zip(download_path, post_href, post_user):
#     post_id = re.search(r'\d+', post_href).group()
#     zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
#     try:
#         with zipfile.ZipFile(zip_filename, 'w') as zipf:
#             for file in os.listdir(download_path):
#                 if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
#                     file_path = os.path.join(download_path, file)
#                     zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

#         # 이미지 파일 삭제
#         for file in os.listdir(download_path):
#             if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
#                 os.remove(os.path.join(download_path, file))
    
#     except Exception as e:
#         print(f"압축 중 오류 발생: {e}")


# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     try:
#         # 페이지의 모든 <a> 태그 찾기
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url.endswith('orig') :
#             # 이미지 URL이 유효한지 확인
#                 clean_url = image_url.split('?')[0]  # 쿼리 매개변수 제거
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))  # 고유한 파일명 생성
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")
                
#                 # 이미지 다운로드
#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")


def get_file_extension(url):
    # URL에서 파일 확장자를 추출합니다.
    path = url.split('?')[0]
    extension = path.split('.')[-1]
    if '/' in extension or len(extension) > 5:
        extension = 'jpg'  # 예외 상황에 대한 기본 확장자 설정
    return extension


# def process_images(driver, post_list, download_path):
#     for href, title, user in post_list:
#         if title != "제목을 찾을 수 없음":
#             print(title)
#             download_images_from_links(driver, href, download_path, title)
#             compress_images_to_zip(download_path,href,user)
    
#     print("_____finished______")

# def download_images_from_links(driver, post_url, download_path, title):
#     driver.get(post_url)
#     time.sleep(2)  # 페이지 로딩 대기

#     failed_downloads = []
#     try:
#         links = driver.find_elements(By.TAG_NAME, 'a')
#         for link in links:
#             image_url = link.get_attribute('href')
#             if image_url and image_url.endswith('orig'):
#                 clean_url = image_url.split('?')[0]
#                 file_extension = get_file_extension(clean_url)
#                 file_name = title + '_' + str(links.index(link))
#                 file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

#                 response = requests.get(image_url, stream=True)
#                 if response.status_code == 200:
#                     with open(file_path, 'wb') as file:
#                         for chunk in response.iter_content(1024):
#                             file.write(chunk)
#                     if os.path.getsize(file_path) < 1024:  # 가정: 파일 크기가 1KB 미만인 경우 비정상
#                         failed_downloads.append((file_name, image_url))
#                 else:
#                     failed_downloads.append((file_name, image_url))
#     except Exception as e:
#         print(f"다운로드 중 오류 발생: {e}")
    
#     return failed_downloads


def download_images_from_links(driver, post_url, download_path, title):
    driver.get(post_url)
    time.sleep(2)  # 페이지 로딩 대기
    i = 0

    failed_downloads = []
    try:
        links = driver.find_elements(By.TAG_NAME, 'a')
        for link in links:
            image_url = link.get_attribute('href')
            if image_url and image_url.endswith('orig'):
                i+=1
                clean_url = image_url.split('?')[0]
                file_extension = get_file_extension(clean_url)
                title = title.replace("/", "_")
                file_name = title + '_' + str(links.index(link))
                file_path = os.path.join(download_path, f"{file_name}.{file_extension}")

                response = requests.get(image_url, stream=True)
                if response.status_code == 200:
                    with open(file_path, 'wb') as file:
                        for chunk in response.iter_content(1024):
                            file.write(chunk)
                    if os.path.getsize(file_path) < 1024:  # 파일 크기 검사
                        failed_downloads.append((file_name, image_url))
                else:
                    failed_downloads.append((file_name, image_url))


        
            
    except Exception as e:
        print(f"다운로드 중 오류 발생: {e}")
#         if i==0:
#             return True
#         else :
#             return False  # 오류 발생시 실패로 간주

    if len(failed_downloads)==0 or i == 0 :
        return True
    else : 
        return False  
    



def compress_images_to_zip(download_path, post_href, post_user):
    post_id = re.search(r'\d+', post_href).group()
    zip_filename = os.path.join(download_path, f"{post_id}_{post_user}.zip")
    
    try:
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in os.listdir(download_path):
                if file.endswith('.jpg') or file.endswith('.png')or file.endswith('.gif') or file.endswith('.webp'):
                    file_path = os.path.join(download_path, file)
                    zipf.write(file_path, os.path.basename(file_path))  # 파일명만 ZIP에 포함

        # 이미지 파일 삭제
        for file in os.listdir(download_path):
            if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.svg') or file.endswith('.gif') or file.endswith('.webp'):
                os.remove(os.path.join(download_path, file))

        # ZIP 파일 크기 확인
        if os.path.getsize(zip_filename) <= 50:  # 50바이트 이하면 실패로 간주
            return False  # 실패 표시
        else:
            return True  # 성공 표시
    except Exception as e:
        print(f"압축 중 오류 발생: {e}")
        return False  # 오류 발생시 실패로 간주

def process_images(driver, post_list, download_path):
    
    duplicate_url = combine_csv_files_remove_duplicates(script_dir)
                      
    failed_posts = []
    i = 1
    total = len(post_list)
    success_url = []
    old_urlsx = duplicate_url.loc[:, 'url'].to_list()
    old_urls = []
    for x in old_urlsx :
        old_urls.append(x.split('?')[0])

    for href, title, user in post_list:
        if title != "제목을 찾을 수 없음":
            if not href.split('?')[0] in old_urls : 
                mm = href.split('?')[0].split('/')[-1]
                print(f"Processing: {total} 중 {i} {mm}__{title}  _  {user}")
                success = download_images_from_links(driver, href, download_path, title)
                is_successful_zip = compress_images_to_zip(download_path, href, user)
                if not is_successful_zip or not success:
                    failed_posts.append((href,title,user))
                    print(f"failed : {href}")
                else :
                    success_url.append([href.split('?')[0], title]) 
            else:
                print(f"중복 : {title}")
        i=i+1

    print("_____finished______")
    print("Failed posts:")
    for url, title, user in failed_posts:
        print(f"User: {user} Title: {title}, URL: {url}")
                      
    if success_url:
        temp = pd.DataFrame(success_url)
        temp.columns = ['url', 'filename']
        temp.to_csv(f"Arca_url_data.csv", index=False)
    else:
        print("No data available to create DataFrame")
    
    return failed_posts



    


In [2]:
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'


options = Options()
options.page_load_strategy = 'eager'  # 'none'으로 설정할 수도 있습니다.
# driver = webdriver.Chrome('/path/to/chromedriver', options=options)

driver = webdriver.Chrome(service=s, options=options)
# 웹드라이버 초기화
# driver = webdriver.Chrome('/Users/jonathanoh/Downloads/chromedriver-mac-arm64/chromedriver')

# 함수 호출 예시
login_to_website(driver, 'bigstone524', 'bigstone524!')



# 이후 스크립트...
post_list = fetch_post_list(driver)[7:]
post_list


#process_images(driver, post_list, download_path)

In [3]:
post_list = []
for i in range(3):
    driver.get(f"https://arca.live/b/aiartreal?p={i+1}")
    if i == 0:
        post_list = post_list+fetch_post_list(driver)[7:]
    else :
        post_list = post_list+fetch_post_list(driver)

post_list.reverse()
post_list

[('https://arca.live/b/aiartreal/95775149?p=3', '요정', '바코드'),
 ('https://arca.live/b/aiartreal/95775208?p=3',
  '반실 테스트 모델 공유 ARAZmixx_020',
  '아리아재'),
 ('https://arca.live/b/aiartreal/95777193?p=3',
  '( hash: a392eacf24, Model:DENG_e9.fp16) 피규어 컨셉,',
  '냥이집사'),
 ('https://arca.live/b/aiartreal/95777691?p=3', 'ARAZmixx_020 찍먹', '현무'),
 ('https://arca.live/b/aiartreal/95777853?p=3',
  'PurelyWholesome_v0.3',
  '이노프리'),
 ('https://arca.live/b/aiartreal/95778241?p=3',
  'Ai 하면서 가장 쾌감을 느끼는 순간이 언제인줄앎?',
  'kkoor'),
 ('https://arca.live/b/aiartreal/95778267?p=3',
  '오프숄더 드레스 (5moon_v10)',
  '너예쁘다'),
 ('https://arca.live/b/aiartreal/95778839?p=3', '오두막 (ARAZmixx_020)', '너예쁘다'),
 ('https://arca.live/b/aiartreal/95779466?p=3',
  '[SFW] hanimix_v19B(nsfw) 찍먹입니다.',
  'ghana'),
 ('https://arca.live/b/aiartreal/95779524?p=3', '수영복', '바코드'),
 ('https://arca.live/b/aiartreal/95779525?p=3',
  '2023년 한 해 내가 그린 그림 진화과정',
  '옵치솜장인'),
 ('https://arca.live/b/aiartreal/95779994?p=3',
  'ARAZmixx_020.fp16 수

In [9]:
list = process_images(driver, post_list, download_path)
# list = process_images(driver, new, download_path)
# list = process_images(driver, [post_list[0]], download_path)

중복 : 요정
중복 : 반실 테스트 모델 공유 ARAZmixx_020
중복 : ( hash: a392eacf24, Model:DENG_e9.fp16) 피규어 컨셉,
중복 : ARAZmixx_020 찍먹
중복 : PurelyWholesome_v0.3
중복 : Ai 하면서 가장 쾌감을 느끼는 순간이 언제인줄앎?
중복 : 오프숄더 드레스 (5moon_v10)
중복 : 오두막 (ARAZmixx_020)
중복 : [SFW] hanimix_v19B(nsfw) 찍먹입니다.
중복 : 수영복
중복 : 2023년 한 해 내가 그린 그림 진화과정
중복 : ARAZmixx_020.fp16 수영복 찍먹입니다.
중복 : PurelyWholesome_v0.2 수영복 찍먹입니다.
중복 : 뉴비에요 잘부탁드립니당
중복 : ARAZmixx_020 찍먹
중복 : hanimix_v19(26D,HQ) 수영복 찍먹입니다.
중복 : 테니스 (스포츠중계 버전)
중복 : 테니스 (일본성장드라마 버전)
중복 : 언더붑 몇장
중복 : 욕조
중복 : 유튜브에 페이트론 링크 다는 사람 있어?
중복 : 피쉬볼 헤드...
중복 : 랜덤 랜덤
중복 : cat(+girl)
중복 : 아침 과제 시작 랜덤짤
중복 : 원했던 건 이게 아닌데!
중복 : 이것좀 해"줘"
중복 : RE : 이것좀 해"줘"
중복 : PurelyWholesome_v0.2.fp16 찍먹.(feat.옵치솜장인 님 로라)
중복 : 5moon 모델 찍먹. (Feat. 옵치솜장인님 로라)
중복 : 브라렛
중복 : 형광드레스
중복 : 에너지 흐름
중복 : 테스트 모델 9번
중복 : 수익화 할 생각을 안하니까 마음은 편하네
Processing: 135 중 36 95801885__원본 이미지 얼굴  _  세이스프스
failed : https://arca.live/b/aiartreal/95801885?p=3
중복 : 5moon_v10 부먹
중복 : hanimix_v19(HQ_3D) 모델 너무 이뻐요..
중복 : ARAZmixx_020 찍먹. (Feat. 옵치솜장인

In [7]:
list

[('https://arca.live/b/aiartreal/95801885?p=3', '원본 이미지 얼굴', '세이스프스'),
 ('https://arca.live/b/aiartreal/95804278?p=3', '올해목표중 독서', '냥이집사'),
 ('https://arca.live/b/aiartreal/95804845?p=3', '반실 그림 몇장', '냥이집사'),
 ('https://arca.live/b/aiartreal/95806994?p=2',
  'hanimix_v19A,B(nsfw) 테스트',
  '달의영혼'),
 ('https://arca.live/b/aiartreal/95807538?p=2', 'PurelyWholesome는', '이노프리'),
 ('https://arca.live/b/aiartreal/95813188?p=2',
  '[PurelyWholesome_v0.3] 오늘의 컨셉은',
  '이노프리'),
 ('https://arca.live/b/aiartreal/95813628?p=2', '배치 돌릴떄', '세이스프스'),
 ('https://arca.live/b/aiartreal/95817885?p=2', '[2D] 메카닉 걸', '야발야발'),
 ('https://arca.live/b/aiartreal/95820496?p=2', '판타지 풍경2', 'Gbox'),
 ('https://arca.live/b/aiartreal/95831262?p=2',
  '[반짝 공유] 아름 - 다희, 하나, 고은',
  '네이티리'),
 ('https://arca.live/b/aiartreal/95831624?p=2',
  'Bikini cowboy_shot (GAMBLE-MIX-REAL-39)',
  'yong91892'),
 ('https://arca.live/b/aiartreal/95832608?p=2',
  '근데 ai그림 여자 생식기 위치가 항상 너무 위 아님?',
  '기모링링'),
 ('https://arca.live/b/aiartrea

In [10]:
len(list)

14

In [6]:
list = process_images(driver, list, download_path)

Processing: 30 중 1 95797786__RE : 이것좀 해"줘"  _  야발야발
Processing: 30 중 2 95801885__원본 이미지 얼굴  _  세이스프스
failed : https://arca.live/b/aiartreal/95801885?p=3
Processing: 30 중 3 95804278__올해목표중 독서  _  냥이집사
failed : https://arca.live/b/aiartreal/95804278?p=3
Processing: 30 중 4 95804367__hanimix_v19A(nsfw) 찍먹합니다.  _  토군
Processing: 30 중 5 95804845__반실 그림 몇장  _  냥이집사
failed : https://arca.live/b/aiartreal/95804845?p=3
Processing: 30 중 6 95806994__hanimix_v19A,B(nsfw) 테스트  _  달의영혼
failed : https://arca.live/b/aiartreal/95806994?p=2
Processing: 30 중 7 95807538__PurelyWholesome는  _  이노프리
failed : https://arca.live/b/aiartreal/95807538?p=2
Processing: 30 중 8 95813188__[PurelyWholesome_v0.3] 오늘의 컨셉은  _  이노프리
failed : https://arca.live/b/aiartreal/95813188?p=2
Processing: 30 중 9 95813628__배치 돌릴떄  _  세이스프스
failed : https://arca.live/b/aiartreal/95813628?p=2
Processing: 30 중 10 95816100__[모델공유] GAMBLE-MIX-REAL  _  yong91892
Processing: 30 중 11 95817885__[2D] 메카닉 걸  _  야발야발
failed : https://arca.live/b/

In [122]:
# compress_and_delete_folders('/Users/jonathanoh/Desktop/참고자료')

Compressed and removed folder: 79349170
Compressed and removed folder: 80052491
Compressed and removed folder: 78772036
Compressed and removed folder: 79365121
Compressed and removed folder: 79373086
Compressed and removed folder: 79278716
Compressed and removed folder: 78848182
Compressed and removed folder: 77808111
Compressed and removed folder: 77848858
Compressed and removed folder: 79329897
Compressed and removed folder: 79459766
Compressed and removed folder: 79920618
Compressed and removed folder: 79460643
Compressed and removed folder: 80452299
Compressed and removed folder: 79319200
Compressed and removed folder: 79835700
Compressed and removed folder: 78002829
Compressed and removed folder: 79435365
Compressed and removed folder: 78323972
Compressed and removed folder: 78922361
Compressed and removed folder: 79190293
Compressed and removed folder: 78911379
Compressed and removed folder: 78906108
Compressed and removed folder: 79933720
Compressed and removed folder: 79181322


In [11]:
temp = []
for url, title, user in list :
    temp.append([url.split('?')[0], title])
df = pd.DataFrame(temp)
df.columns=['url', 'filename']
# script_dir = '/Users/jonathanoh/Desktop/script/crawling'
csv_fail_fiile = os.path.join(script_dir, 'fail_list.csv')
df.to_csv(csv_fail_fiile, index=False)
df = combine_csv_files_remove_duplicates(script_dir)

In [12]:
df = combine_csv_files_remove_duplicates(script_dir)

In [13]:
df

,url,filename
0,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4931442.jpeg
1,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4924345.jpeg
2,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926329.jpeg
3,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926315.jpeg
4,https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...,civitai_Day_4926328.jpeg
...,...,...
4282,https://arca.live/b/aiartreal/95831624,Bikini cowboy_shot (GAMBLE-MIX-REAL-39)
4283,https://arca.live/b/aiartreal/95832608,근데 ai그림 여자 생식기 위치가 항상 너무 위 아님?
4284,https://arca.live/b/aiartreal/95836538,YA짤
4285,https://arca.live/b/aiartreal/95841469,ㅁㅁ하면... 스트레스 쌓이는군요...


In [11]:
k = [list[5]]
del list[5]

In [52]:
df['url'] = df['url'].apply(lambda x: x.split('?')[0])

In [53]:
df.to_csv(os.path.join(script_dir, "url_datas.csv"), index=False)

<bound method NDFrame.head of                                                     url  \
0     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
1     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
2     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
3     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
4     https://image.civitai.com/xG1nkqKTMzGDvpLrqFT7...   
...                                                 ...   
7226             https://arca.live/b/aiartreal/95066061   
7227             https://arca.live/b/aiartreal/95053202   
7228             https://arca.live/b/aiartreal/95052445   
7229             https://arca.live/b/aiartreal/95074503   
9718             https://arca.live/b/aiartreal/95074892   

                                               filename  
0                              civitai_Year_127497.jpeg  
1                              civitai_Year_175719.jpeg  
2                              civitai_Year_295008.jpeg  
3                            